In [26]:
using LinearAlgebra
using Polyhedra

using CDDLib, Clarabel

using TropicalFrechetMeans

In [14]:
function trop_normalize(x)
    return x .- first(x)
end

trop_normalize (generic function with 1 method)

In [15]:
using JSON3
using DataFrames

# Function to read the JSON file and convert to a list of matrices
function read_and_convert_json(file_path::String)
    # Read the JSON file
    json_data = JSON3.read(file_path)
    
    # Extract elements from the nested arrays
    elements = [x[1] for x in json_data]
    elements = rationalize.(10000 * elements, tol=1e-2)
    
    # Convert elements into matrices
    num_elements = length(elements)
    matrices = []
    
    for i in 1:64:num_elements
        # Get the next 64 elements
        matrix_elements = elements[i:min(i+63, num_elements)]
        
        # Convert to an 8x8 matrix if there are 64 elements, otherwise create a smaller matrix
        matrix_size = length(matrix_elements)
        sqrt_size = Int(sqrt(matrix_size))
        push!(matrices, reshape(matrix_elements, sqrt_size, sqrt_size))
    end
    
    return matrices
end

# Read and convert the JSON file
file_path = "all_matrices.json"
matrices = map(read_and_convert_json(file_path)) do m
    Rational{BigInt}.(m)
end
taxa = ["Tg", "Et", "Cp", "Ta", "Bb", "Tt", "Pv", "Pf"]

8-element Vector{String}:
 "Tg"
 "Et"
 "Cp"
 "Ta"
 "Bb"
 "Tt"
 "Pv"
 "Pf"

In [16]:
"""
Take a matrix of pairwise distances between taxa and returns the cophenetic vector.
"""
function cophenetic_from_distance(pairwise)
    n = size(pairwise, 1)
    coph = [pairwise[i, j] for i in 1:n-1 for j in i+1:n]
    return coph
end

cophenetic_from_distance

In [17]:
"""
Check if a distance matrix defines a phylogenetic tree
"""
function is_phylogenetic_tree(D)
    n = size(D, 1)
    
    # Check if the matrix is symmetric and non-negative
    for i in 1:n
        for j in i:n
            if D[i, j] != D[j, i] || D[i, j] < 0
                return false
            end
        end
    end

    # Check the four-point condition
    for i in 1:n-3
        for j in i+1:n-2
            for k in j+1:n-1
                for l in k+1:n
                    # Calculate distances
                    D_ij_kl = D[i, j] + D[k, l]
                    D_ik_jl = D[i, k] + D[j, l]
                    D_il_jk = D[i, l] + D[j, k]
                    
                    # Check the four-point condition
                    if !(D_ij_kl >= D_ik_jl && D_ij_kl >= D_il_jk) &&
                       !(D_ik_jl >= D_ij_kl && D_ik_jl >= D_il_jk) &&
                       !(D_il_jk >= D_ij_kl && D_il_jk >= D_ik_jl)
                        return false
                    end
                end
            end
        end
    end
    
    return true
end

is_phylogenetic_tree

In [18]:
"""
Check if a distance matrix defines an ultrametric tree
"""
function is_ultrametric_tree(D)
    n = size(D, 1)
    
    # Check if the matrix is symmetric and non-negative
    for i in 1:n
        for j in i:n
            if D[i, j] != D[j, i] || D[i, j] < 0
                return false
            end
        end
    end

    # Check the ultrametric condition
    for i in 1:n-2
        for j in i+1:n-1
            for k in j+1:n
                # Calculate distances
                Dij = D[i, j]
                Dik = D[i, k]
                Djk = D[j, k]
                
                # Check if the largest distance is attained at least twice
                if !(Dij <= max(Dik, Djk) && Dik <= max(Dij, Djk) && Djk <= max(Dij, Dik))
                    return false
                end
            end
        end
    end
    
    return true

end

is_ultrametric_tree

In [19]:
println("All phylogenetic: ", all([is_phylogenetic_tree(matrices[i]) for i = 1:268]))
println("All ultrametric: ", all([is_ultrametric_tree(matrices[i]) for i = 1:268]))

All phylogenetic: true
All ultrametric: false


In [20]:
coph_vecs = [cophenetic_from_distance(mat) for mat in matrices]

268-element Vector{Vector{Rational{BigInt}}}:
 [3784, 6626, 9906, 6521, 11778, 8750, 7661, 7601, 10881, 7496  …  4901, 15579, 12551, 11462, 12194, 9167, 8078, 14217, 13128, 1089]
 [3485, 8484, 9427, 8865, 25257, 9257, 10300, 8814, 9756, 9195  …  3645, 25010, 9010, 10054, 24449, 8449, 9492, 20033, 21076, 2163]
 [3570, 5592, 6890, 4591, 4849, 5136, 4872, 3905, 6062, 3763  …  6609, 6867, 7154, 6890, 2383, 4367, 4103, 4625, 4361, 274]
 [1696, 4297, 5665, 6408, 4431, 4218, 4248, 4473, 5841, 6584  …  9039, 7062, 6849, 6880, 5542, 6984, 7014, 5006, 5037, 557]
 [1864, 8173, 6779, 10056, 10843, 9010, 9319, 8433, 7039, 10315  …  11759, 12546, 10713, 11022, 15116, 13283, 13592, 12332, 12642, 2722]
 [3801, 11567, 5222, 5725, 5642, 8167, 7548, 12836, 6491, 6994  …  6129, 6046, 8571, 7952, 601, 7410, 6791, 7327, 6708, 4443]
 [751, 29706, 6877, 4376, 5857, 9999, 8966, 29899, 7070, 4569  …  6079, 7560, 11702, 10669, 3359, 8305, 7272, 9787, 8754, 5104]
 [4080, 8200, 9699, 9924, 11942, 9302, 9463, 7689,

In [24]:
@time tropical_frechet_mean(coph_vecs[1]) |> trop_normalize

MethodError: MethodError: no method matching tropical_frechet_mean(::Vector{Rational{BigInt}})

Closest candidates are:
  tropical_frechet_mean(!Matched::Type{Opt}, !Matched::Any; power) where Opt<:MathOptInterface.AbstractOptimizer
   @ TropicalFrechetMeans ~/Repos/TropicalFrechetMeans.jl/src/models.jl:74


In [25]:
@time phylo_frech = tropical_frechet_set(coph_vecs[1])

MethodError: MethodError: no method matching tropical_frechet_set(::Vector{Rational{BigInt}})

Closest candidates are:
  tropical_frechet_set(!Matched::Type{Opt}, !Matched::Array{Vector{T}, 1}; power, tol) where {Opt<:MathOptInterface.AbstractOptimizer, T<:Real}
   @ TropicalFrechetMeans ~/Repos/TropicalFrechetMeans.jl/src/polyhedra.jl:79
  tropical_frechet_set(!Matched::Type{Opt}, !Matched::Lib, !Matched::Array{Vector{T}, 1}; power, tol) where {Opt<:MathOptInterface.AbstractOptimizer, Lib<:Polyhedra.Library, T<:Real}
   @ TropicalFrechetMeans ~/Repos/TropicalFrechetMeans.jl/src/polyhedra.jl:70


In [23]:
TropicalFrechetMeans.conjugate_gradients(coph_vecs)

Rational{BigInt}[3107905//134, 8181165//268, 3037669//134, 7165219//268, 6381613//268, 3669481//134, 7185971//268, 5897589//268, 973659//67, 5013745//268, 1060449//67, 2600717//134, 5048431//268, 4558045//268, 91545//4, 5403973//268, 1597939//67, 6238753//268, 1767277//134, 3028411//268, 2034739//134, 3916479//268, 53897787//4556, 1216265//67, 1178015//67, 1903027//134, 3654461//268, 91459151//9112]2//3
Rational{BigInt}[3108106//201 2726921//134 3037669//201 7165219//402 6381613//402 3669481//201 7185971//402 1965863//134 649106//67 5013745//402 706966//67 2600717//201 5048431//402 4558045//402 30515//2 5403973//402 3195878//201 6238753//402 1767277//201 3028411//402 2034739//201 1305493//134 17965929//2278 2432530//201 2356030//201 1903027//201 3654461//402 91459151//13668; 3108106//201 2727055//134 3037468//201 7165219//402 6381613//402 3669481//201 7185971//402 1965863//134 649106//67 5013745//402 706966//67 2600717//201 5048431//402 4558045//402 30515//2 5403973//402 3195878//201 6

Excessive output truncated after 524292 bytes.

 1965863//268 324553//67 5013745//804 353483//67 2600717//402 5048431//804 4558045//804 30515//4 5403973//804 1597939//201 6238753//804 1767277//402 3028411//804 2034739//402 1305493//268 17965929//4556 1216265//201 1178216//201 1903027//402 3654461//804 91431815//27336; 1553852//201 2726921//268 3037669//402 7165219//804 6381613//804 3669481//402 7185971//804 1965863//268 324553//67 5013745//804 353483//67 2600717//402 5048431//804 4558045//804 30515//4 5403973//804 1597939//201 6238753//804 1767277//402 3028411//804 2034739//402 1305493//268 17965929//4556 1216265//201 1178015//201 1903429//402 3654461//804 91459151//27336; 1554053//201 2726653//268 3037669//402 7165219//804 6381613//804 3669481//402 7185971//804 1965863//268 324553//67 5013745//804 353483//67 2600717//402 5048431//804 4558045//804 30515//4 5403973//804 1597939//201 6238753//804 1767277//402 3028411//804 2034739//402 1305493//268 17965929//4556 1216265//201 1178015//201 1903429//402 3654461//804 91459151//27336; 1554

28-element Vector{Rational{BigInt}}:
  1554254//201
  2726653//268
  3037669//402
  7165219//804
  6381613//804
  3669481//402
  7185971//804
  1965863//268
   324553//67
  5013745//804
         ⋮
  3028411//804
  2034739//402
  1305493//268
 17965929//4556
  1216265//201
  1178015//201
  1903027//402
  3654461//804
 91459151//27336